In [1]:
import pandas as pd
import gurobipy as gp
import numpy as np
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
import time

In [2]:
%matplotlib notebook

In [3]:
df = pd.read_csv('/Users/nguyenhalinh/Desktop/Research/dataset.csv')
df = df[df['state_code'] == 'PA']
df

/var/folders/vf/c31mrqv54zg80c74f5bw2__c0000gn/T/ipykernel_52871/3839772177.py:1: DtypeWarning: Columns (22,23,24,26,27,28,29,30,31,32,33,38,43,44) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('/Users/nguyenhalinh/Desktop/Research/dataset.csv')


,activity_year,lei,derived_msa-md,state_code,county_code,census_tract,conforming_loan_limit,derived_loan_product_type,derived_dwelling_category,derived_ethnicity,...,denial_reason-2,denial_reason-3,denial_reason-4,tract_population,tract_minority_population_percent,ffiec_msa_md_median_family_income,tract_to_msa_income_percentage,tract_owner_occupied_units,tract_one_to_four_family_homes,tract_median_age_of_housing_units
0,2021,54930034MNPILHP25H80,14100,PA,42093.0,4.209305e+10,C,Conventional:First Lien,Single Family (1-4 Units):Site-Built,Not Hispanic or Latino,...,NaN,NaN,NaN,4686,9.80,76500,74,1035,2246,71
1,2021,54930034MNPILHP25H80,10900,PA,42095.0,4.209501e+10,C,Conventional:First Lien,Single Family (1-4 Units):Site-Built,Ethnicity Not Available,...,NaN,NaN,NaN,7567,35.09,84900,78,2061,2963,64
2,2021,54930034MNPILHP25H80,39740,PA,42011.0,4.201101e+10,C,Conventional:First Lien,Single Family (1-4 Units):Site-Built,Not Hispanic or Latino,...,NaN,NaN,NaN,3040,3.39,82200,120,1032,1278,38
3,2021,54930034MNPILHP25H80,33874,PA,42091.0,4.209120e+10,C,Conventional:First Lien,Single Family (1-4 Units):Site-Built,Not Hispanic or Latino,...,NaN,NaN,NaN,4006,9.84,116300,104,1232,1319,53
4,2021,54930034MNPILHP25H80,37964,PA,42101.0,4.210103e+10,C,Conventional:First Lien,Single Family (1-4 Units):Site-Built,Not Hispanic or Latino,...,NaN,NaN,NaN,4514,98.54,67500,70,998,1677,76
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
861410,2021,54930034MNPILHP25H80,33874,PA,42091.0,4.209120e+10,C,Conventional:First Lien,Single Family (1-4 Units):Site-Built,Not Hispanic or Latino,...,NaN,NaN,NaN,2560,14.22,116300,111,853,1049,46
861411,2021,54930034MNPILHP25H80,39740,PA,42011.0,4.201101e+10,C,Conventional:First Lien,Single Family (1-4 Units):Site-Built,Not Hispanic or Latino,...,NaN,NaN,NaN,6790,14.04,82200,91,1952,2395,46
861412,2021,54930034MNPILHP25H80,37964,PA,42045.0,4.204541e+10,C,Conventional:First Lien,Single Family (1-4 Units):Site-Built,Not Hispanic or Latino,...,NaN,NaN,NaN,4317,5.19,67500,181,1400,1577,59
861413,2021,54930034MNPILHP25H80,37964,PA,42101.0,4.210104e+10,C,Conventional:First Lien,Single Family (1-4 Units):Site-Built,Not Hispanic or Latino,...,NaN,NaN,NaN,6246,24.37,67500,141,1964,2246,48


In [4]:
extracted_df = df[['lei', 'census_tract', 'county_code','action_taken', 'loan_amount', 'property_value', 'income', 'debt_to_income_ratio','loan_term', 'denial_reason-1', 'denial_reason-2', 'loan_purpose','applicant_race-1','applicant_sex']]
# extracted_df = df
extracted_df = extracted_df[extracted_df['action_taken'].isin([1,3])] 
extracted_df = extracted_df[extracted_df['loan_purpose'] == 1]
#clean property value column
extracted_df = extracted_df[extracted_df['property_value'].notna()]
extracted_df['property_value'] = np.where(extracted_df['property_value'] == 'Exempt',1111, extracted_df['property_value'])
extracted_df["property_value"] = pd.to_numeric(extracted_df["property_value"])
extracted_df["property_value"] = extracted_df["property_value"]/1000

extracted_df["loan_amount"] = extracted_df["loan_amount"]/1000


# clean loan terms column
extracted_df['loan_term'] = np.where(extracted_df['loan_term'] == 'Exempt',1111, extracted_df['loan_term'])
extracted_df['loan_term'] = pd.to_numeric(extracted_df['loan_term'])

In [5]:
#clean debt to income ratio column
extracted_df['debt_to_income_ratio'] = np.where(extracted_df['debt_to_income_ratio'] == '<20%',10, extracted_df['debt_to_income_ratio'])
extracted_df['debt_to_income_ratio'] = np.where(extracted_df['debt_to_income_ratio'] == '20%-<30%',25, extracted_df['debt_to_income_ratio'])
extracted_df['debt_to_income_ratio'] = np.where(extracted_df['debt_to_income_ratio'] == '30%-<36%',33, extracted_df['debt_to_income_ratio'])
extracted_df['debt_to_income_ratio'] = np.where(extracted_df['debt_to_income_ratio'] == '50%-60%',55, extracted_df['debt_to_income_ratio'])
extracted_df['debt_to_income_ratio'] = np.where(extracted_df['debt_to_income_ratio'] == '>60%',60, extracted_df['debt_to_income_ratio'])
extracted_df['debt_to_income_ratio'] = np.where(extracted_df['debt_to_income_ratio'] == 'Exempt',1111, extracted_df['debt_to_income_ratio'])
extracted_df["debt_to_income_ratio"] = pd.to_numeric(extracted_df["debt_to_income_ratio"])

# extracted_df

In [6]:
extracted_df = extracted_df[extracted_df['property_value']!= 1111]
extracted_df = extracted_df[extracted_df['debt_to_income_ratio']!= 1111]
extracted_df = extracted_df[extracted_df['loan_term']!= 1111]

extracted_df.loc[extracted_df['action_taken'] == 1, 'outcome'] = 1
extracted_df.loc[extracted_df['action_taken'] == 3, 'outcome'] = 0

In [7]:
extracted_df = extracted_df[extracted_df['debt_to_income_ratio'].notna()]
extracted_df = extracted_df[extracted_df['loan_amount'].notna()]
extracted_df = extracted_df[extracted_df['income'].notna()]
extracted_df = extracted_df[extracted_df['property_value'].notna()]
extracted_df

,lei,census_tract,county_code,action_taken,loan_amount,property_value,income,debt_to_income_ratio,loan_term,denial_reason-1,denial_reason-2,loan_purpose,applicant_race-1,applicant_sex,outcome
2,54930034MNPILHP25H80,4.201101e+10,42011.0,1,375.0,395.0,153.0,25.0,360.0,10,NaN,1,5.0,1,1.0
3,54930034MNPILHP25H80,4.209120e+10,42091.0,1,415.0,585.0,146.0,40.0,360.0,10,NaN,1,5.0,2,1.0
4,54930034MNPILHP25H80,4.210103e+10,42101.0,1,115.0,115.0,51.0,44.0,360.0,10,NaN,1,3.0,1,1.0
6,54930034MNPILHP25H80,4.201711e+10,42017.0,1,315.0,355.0,70.0,25.0,360.0,10,NaN,1,2.0,2,1.0
7,54930034MNPILHP25H80,4.208930e+10,42089.0,3,185.0,175.0,61.0,43.0,360.0,4,NaN,1,3.0,1,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
861394,54930034MNPILHP25H80,4.212980e+10,42129.0,1,305.0,325.0,67.0,33.0,360.0,10,NaN,1,5.0,1,1.0
861397,54930034MNPILHP25H80,4.200760e+10,42007.0,1,225.0,225.0,138.0,33.0,360.0,10,NaN,1,5.0,2,1.0
861398,54930034MNPILHP25H80,4.209120e+10,42091.0,1,295.0,385.0,175.0,42.0,360.0,10,NaN,1,6.0,1,1.0
861401,54930034MNPILHP25H80,4.201101e+10,42011.0,1,195.0,195.0,98.0,33.0,360.0,10,NaN,1,5.0,1,1.0


## CORRELATION COEFFICIENT


I will calculate correlation of each variable (loan_amount, property_value, debt_to_income_ratio, and income) with target variable (outcome) using Point-Biserial correlation

In [8]:
import scipy.stats as stats
import pylab

from scipy.stats import pointbiserialr

In [9]:
def pbc_cal(dataframe, x, y):
    """ A function to calculate Point-Biserial Correlation"""
#     get continuous & binary data
    x_array = dataframe[x] #binary
    y_array = dataframe[y]

    pbc = pointbiserialr(x_array,y_array)
    return pbc[0], pbc[1]

In [10]:
from pandas.api.types import is_numeric_dtype

matrix = {'variable': [], "correlation" : [], "p-val":[]}
for col in extracted_df:
    if (is_numeric_dtype(extracted_df[col]) == True):
        notna_df = extracted_df[extracted_df[col].notna()]
        pbc, pval = pbc_cal(notna_df, 'outcome', col)
        matrix['variable'].append(col)
        matrix['correlation'].append(pbc)
        matrix['p-val'].append(pval)

/opt/anaconda3/envs/pyclass/lib/python3.9/site-packages/scipy/stats/stats.py:4023: PearsonRConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  warnings.warn(PearsonRConstantInputWarning())


## DEA SCORE

In [19]:
class DEA_Score:
    """ A class to calculate DEA score and get the weights for each application"""
    
    def __init__(self, df, number_applications, correlation):
        """
        df: dataframe of all applications used to calculate DEA score
        number_applications: number of applications in the dataframe needed to calculate DEA score
        correlation: Point-Biserial correlation dictionary where the key is the variable name and value is correlation
        """
        self.number_applications = number_applications
        self.df = df.head(self.number_applications)
        self.r_val = correlation



    def DEA(self, app_index):
        """ A function to calculate DEA score using optimization
            - app_index: the index of application to be optimized
            - Return DEA score and a list of weights for that application
        """
        env = gp.Env(empty=True)
        env.setParam('OutputFlag',0)
        env.start()
        model = gp.Model()

        
        time_before = time.time()
        
        weights = []

        E = model.addVar(ub=1)
        w = model.addVars(self.number_applications)
        
        for variable in self.r_val:
            if matrix[variable] < 0: #if correlation coefficient of the variable is <0, the variable is input
                model.addConstr(gp.quicksum(self.df[variable][j]*w[j] for j in range(self.number_applications)) <= self.df[variable][app_index]*E )
            else: #else, the variable is output
                model.addConstr(gp.quicksum(self.df[variable][j]*w[j] for j in range(self.number_applications))  >= self.df[variable][app_index])

    
        #sum of weights equal 1
        model.addConstr(gp.quicksum(w[j] for j in range(self.number_applications))  == 1)

        model.setObjective(E)
        model.optimize()

        weights = [w[i].x for i in range(self.number_applications)]
        
        time_after = time.time()
        print("TIME RUN: " + str(time_after - time_before))

        return model.ObjVal,weights

    
    def new_data(self):
        """ A function to get all the DEA scores and weights of the dataframe"""
        weights = {}
        dea_list = []
         
        for i in range (self.number_applications): #number of applications
            dea, weight = self.DEA(i)
            dea_list.append(dea)
            print("Processing DMU %i, Efficiency score %.3f" % (i, dea))
            for j in range (self.number_applications):
                if 'weights' + str(j) not in weights:
                    weights['weights' + str(j)] = [weight[j]]
                else:
                    weights['weights' + str(j)].append(weight[j])  
                    
        return dea_list, weights

In [21]:
#Get census tract data 
# census_list = extracted_df[extracted_df['county_code'] == 42011]['census_tract'].tolist()
# census_df = extracted_df[extracted_df['census_tract'].isin(census_list[:5])]
# 42091201204
# 42091207400
# 42001030400
county_df = extracted_df[extracted_df['county_code'] == 42011]

county_df.reset_index(inplace = True, drop = True)


In [22]:
#Calculate correlation coefficient based on census tract data
from pandas.api.types import is_numeric_dtype

matrix = {}
for col in ['loan_amount', 'property_value', 'income', 'debt_to_income_ratio']:
    if (is_numeric_dtype(county_df[col]) == True):
        notna_df = census_df[county_df[col].notna()]
        pbc, pval = pbc_cal(notna_df, 'outcome', col)
        matrix[col] = pbc
        
matrix

{'loan_amount': 0.1353806020048105,
 'property_value': 0.12325030699222642,
 'income': 0.0707741287522641,
 'debt_to_income_ratio': -0.1862727759234583}

In [ ]:
DEA1 = DEA_Score(census_df,census_df.shape[0], matrix)

DEA_score = DEA1.new_data()[0]
weights = DEA1.new_data()[1]

Set parameter Username
Academic license - for non-commercial use only - expires 2023-08-21
Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (mac64[x86])
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 5 rows, 6090 columns and 30427 nonzeros
Model fingerprint: 0x88dc973c
Coefficient statistics:
  Matrix range     [1e+00, 2e+03]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 4e+02]

Concurrent LP optimizer: dual simplex and barrier
Showing barrier log only...

Presolve removed 0 rows and 188 columns
Presolve time: 0.03s
Presolved: 5 rows, 5902 columns, 29487 nonzeros

Ordering time: 0.00s

Barrier statistics:
 AA' NZ     : 1.000e+01
 Factor NZ  : 1.500e+01 (roughly 2 MB of memory)
 Factor Ops : 5.500e+01 (less than 1 second per iteration)
 Threads    : 1

Barrier performed 0 iterations in 0.04 seconds (0.01 work units)
Barrier solve interrupted - model solved by another algorithm


Solved with

In [ ]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)


dea_county_df = county_df[['loan_amount', 'property_value', 'income','debt_to_income_ratio', 'outcome']].copy()

dea_county_df['DEA'] = DEA_score
for j in range(census_df.shape[0]):
    dea_county_df['weights' + str(j)] = weights['weights' + str(j)]
dea_county_df

## Efficient Frontier

In [ ]:
from matplotlib.ticker import MaxNLocator
from matplotlib import cm
from mpl_toolkits.mplot3d import Axes3D

def plot_scatter(criteria_1, criteria_2,  criteria_3):
    """ A function to draw 3D curve to visualize the efficient frontier """

    #Filter all applications that are determined as efficient (DEA score = 1)
    df1 = dea_county_df[dea_county_df['DEA'] == 1]
#     df1 = dea_census_df.filter(items=[50,56,34], axis=0)
# 19,27,32
# 27,8,32

    #Filter all applications that are determined as inefficient (DEA score != 1)
    df2 = dea_county_df[dea_county_df['DEA'] != 1]

    #Get the points of efficient applications
    columns = dea_county_df.columns
    c1, c2, c3 = columns[criteria_1], columns[criteria_2],columns[criteria_3]
    df1_selected_array = df1[[c1,c2,c3]].to_numpy()
    
    fig = plt.figure()
    ax1 = Axes3D(fig)
    
    #Set label
    ax1.set_xlabel(str(c1)) 
    ax1.set_ylabel(str(c2)) 
    ax1.set_zlabel(str(c3))
    
    #Set axis
    Xs = df1_selected_array[:,0]
    Ys = df1_selected_array[:,1]
    Zs = df1_selected_array[:,2]

    
    for i in range(len(dea_census_df)):
        if i in df1.index:
            ax1.scatter(xs= df1[c1][i], ys =  df1[c2][i], zs= df1[c3][i], c ='r')
            ax1.text(dea_census_df[c1][i], dea_census_df[c2][i], dea_census_df[c3][i],  '%s' % (str(i)), size=10, zorder=1,  color='r') 
        else:
            ax1.scatter(xs= df2[c1][i], ys =  df2[c2][i], zs= df2[c3][i], c ='blue')
            
#             ax1.text(dea_census_df[c1][i], dea_census_df[c2][i], dea_census_df[c3][i],  '%s' % (str(i)), size=10, zorder=1,  color='DarkBlue')
    
#     surf = ax1.plot_trisurf(Xs, Ys, Zs, linewidth=0)
#     fig.colorbar(surf)

    ax1.xaxis.set_major_locator(MaxNLocator(5))
    ax1.yaxis.set_major_locator(MaxNLocator(6))
    ax1.zaxis.set_major_locator(MaxNLocator(5))

    fig.tight_layout()

    ax1.invert_xaxis()
    plt.show()

#### 0: loan amount
#### 1: property value
#### 2: income
#### 3: debt

In [ ]:
plot_scatter(2,0,1)

In [ ]:
def plot_scatter2D(criteria_1, criteria_2):
    """A function to draw 2D curve to visualize the efficient frontier """

    df1 = dea_census_df[dea_census_df['DEA'] == 1]
    df2 = dea_census_df[dea_census_df['DEA'] != 1]
    ax1 = df2.plot.scatter(x=criteria_1, y=criteria_2, c='DarkBlue')
    ax2 = df1.plot(x= criteria_1, y= criteria_2, kind='scatter', ax=ax1, c='r', label=1)
    
    c1, c2 = dea_census_df.keys()[criteria_1], dea_census_df.keys()[criteria_2]
    for i in range(len(dea_census_df)):
        if i in df1.index:
            ax2.annotate(i, (dea_census_df[c1][i]-7.5, dea_census_df[c2][i]), color='r')
        else:
            ax1.annotate(i, (dea_census_df[c1][i], dea_census_df[c2][i]), color='DarkBlue')

    
    return ax2

In [ ]:
# plot_scatter2D(2,0)

In [ ]:
# plot_scatter2D(1,0)

In [ ]:
# plot_scatter2D(1,2)